In [1]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
np.random.seed(0)

In [2]:
cell_df = pd.read_csv('image_bins_GLCM_all.csv')
cell_df.head()

,var,asm,contrast,correlation,idm,entropy,homogenity,energy,dissimilarity,Class
0,2378.244684,0.009158,1211.821712,0.797017,0.525642,7.810224,0.525642,0.095697,8.398868,1
1,2231.711952,0.008901,1225.951925,0.784520,0.493865,7.915158,0.493865,0.094346,8.767637,1
2,2054.426557,0.006830,1165.783478,0.778996,0.464011,8.416442,0.464011,0.082646,8.621760,1
3,1567.735710,0.008009,668.185088,0.824332,0.499687,8.325647,0.499687,0.089494,6.199938,1
4,1557.136176,0.004084,890.222724,0.777694,0.468373,9.291812,0.468373,0.063908,7.383222,1


In [3]:
feature_df = cell_df[['var','asm','contrast','correlation','idm','entropy','homogenity','energy','dissimilarity']]
X = np.asarray(feature_df)
y = np.asarray(cell_df['Class'])

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 4)

In [5]:
#16800 rows x 8 columns for training IV
X_train.shape

#16800 rows x 1 columns for training DV
y_train.shape

#7200 rows x 8 columns for testing IV
X_test.shape

#7200 rows x 1 columns for testing DV
y_test.shape

(7200,)

In [6]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)

In [7]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8820833333333333


In [8]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.88      0.88      3531
           1       0.89      0.88      0.88      3669

    accuracy                           0.88      7200
   macro avg       0.88      0.88      0.88      7200
weighted avg       0.88      0.88      0.88      7200



In [10]:
from sklearn import svm, metrics
from sklearn.metrics import confusion_matrix
import seaborn as sns
print(confusion_matrix(y_test, y_pred))

print('Accuracy: {0:.3f}'.format(metrics.accuracy_score(y_test, y_pred)))
print('F1 Score: {0:.3f}'.format(metrics.f1_score(y_test, y_pred)))
print('Sensitivity: {0:.3f}'.format(metrics.recall_score(y_test, y_pred)))
print('Precision: {0:.3f}'.format(metrics.precision_score(y_test, y_pred)))
print('Recall: {0:.3f}'.format(metrics.recall_score(y_test, y_pred)))

[[3114  417]
 [ 432 3237]]
Accuracy: 0.882
F1 Score: 0.884
Sensitivity: 0.882
Precision: 0.886
Recall: 0.882


In [22]:
def getbins(cell_image):
        a = np.array(cell_image[:,:,0], dtype = np.uint8)
        d = dict()
        for i in range (0,256):
            d[i]=0

        x = []
        for i in range (0,256):
            x.append(i)    

        y = []
        for i in range (0,256):
            y.append(d[i])
            
        num = 0
        den = 0
        for i in a.flatten():
            d[i] = d.get(i,0) + 1

        b = 0
        for i in x:
            b = d.get(i,0) + 1
            num = num + b*i
            den = den + b
        cogr = num / den
        
        a = np.array(cell_image[:,:,1], dtype = np.uint8)
        d = dict()
        for i in range (0,256):
            d[i]=0

        x = []
        for i in range (0,256):
            x.append(i)    

        y = []
        for i in range (0,256):
            y.append(d[i])

        num = 0
        den = 0
        for i in a.flatten():
            d[i] = d.get(i,0) + 1

        b = 0
        for i in x:
            b = d.get(i,0) + 1
            num = num + b*i
            den = den + b
            cogg = num / den
        
        a = np.array(cell_image[:,:,2], dtype = np.uint8)
        d = dict()
        for i in range (0,256):
            d[i]=0

        x = []
        for i in range (0,256):
            x.append(i)    

        y = []
        for i in range (0,256):
            y.append(d[i])

        num = 0
        den = 0
        for i in a.flatten():
            d[i] = d.get(i,0) + 1

        b = 0
        for i in x:
            b = d.get(i,0) + 1
            num = num + b*i
            den = den + b
        cogb = num / den
        
        h, w, _ = cell_image.shape
        R, G, B = 0, 1, 2

        for color, cog in [(R, cogr), (G, cogg), (B, cogb)]:
            for r in range(h):
                for c in range(w):
                    if cell_image[r, c, color] <= cog:
                        cell_image[r, c, color] = 0
                    else:
                        cell_image[r, c, color] = 1
        
        bins0 = 0
        bins1 = 0
        bins2 = 0
        bins3 = 0
        bins4 = 0
        bins5 = 0
        bins6 = 0
        bins7 = 0
        for i in range(h):
            for j in range(w):
                r= cell_image[i,j,0];
                g= cell_image[i,j,1];
                b= cell_image[i,j,2];
        
                if r==0 and g==0 and b==0:
                        bins0= bins0 + 1
                elif r==0 and g==0 and b==1:
                        bins1= bins1 + 1
                elif r==0 and g==1 and b==0:
                        bins2= bins2 + 1
                elif r==0 and g==1 and b==1:
                        bins3= bins3 + 1
                elif r==1 and g==0 and b==0:
                        bins4= bins4 + 1
                elif r==1 and g==0 and b==1:
                        bins5= bins5 + 1
                elif r==1 and g==1 and b==0:
                        bins6= bins6 + 1
                elif r==1 and g==1 and b==1:
                        bins7= bins7 + 1
                        
        l=[]
        l.extend([bins0, bins1, bins2, bins3, bins4, bins5, bins6, bins7])
        
        return l

In [25]:
from tkinter import *
from tkinter import ttk
from tkinter import filedialog
import numpy as np
import cv2

class Root(Tk):
    def __init__(self):
        super(Root, self).__init__()
        self.title("Bins Approach with Random Forest Classifier")
        self.minsize(640, 400)
        self.labelFrame = ttk.LabelFrame(self, text = "Open File")
        self.labelFrame.grid(column = 0, row = 1, padx = 20, pady = 20)
        self.button()
        self.button1() 
 
    def button(self):
        self.button = ttk.Button(self.labelFrame, text = "Browse A File",command = self.fileDialog)
        self.button.grid(column = 1, row = 1)
  
    def fileDialog(self):
        self.filename = filedialog.askopenfilename(initialdir =  "/", title = "Select A File", filetype =
        (("png files","*.png"),("all files","*.*")) )
        self.label = ttk.Label(self.labelFrame, text = "")
        self.label.grid(column = 1, row = 2)
        self.label.configure(text = self.filename)
 
    def button1(self):
        self.button = ttk.Button(self.labelFrame, text = "submit", command = self.get_prediction)
        self.button.grid(column = 1, row = 20)
        
    
    def get_prediction(self):
        my_image = cv2.imread(self.filename)
        dst = cv2.fastNlMeansDenoisingColored(my_image,None,10,10,7,29)
        h = getbins(dst)
        y=clf.predict([h])
        if(y==1):
            self.label.configure(text="The Cell Image is Parasitic")
        else:
            self.label.configure(text="The Cell Image is Non-Parasitic") 
root = Root()
root.mainloop() 